# fz-Serpent Plugin Example Usage

This notebook demonstrates how to use the fz-Serpent plugin for parametric studies with the Serpent Monte Carlo code.

The examples cover:
1. Installing dependencies (fz framework and serpentTools)
2. Downloading example input files
3. Parsing input files to identify variables
4. Compiling input files with specific parameter values
5. Running calculations (requires Serpent installation)
6. Parsing Serpent results with serpentTools

**Note:** Running actual Serpent calculations requires a Serpent installation and nuclear data libraries.

## Setup: Install Dependencies

First, install the Funz framework and serpentTools for result parsing.

In [ ]:
# Install fz framework from GitHub
%pip install -q git+https://github.com/Funz/fz.git

# Install serpentTools for result parsing
%pip install -q serpentTools

# Install fz-Serpent plugin
import fz
fz.install("Serpent")

print("✓ Dependencies installed successfully!")

## Setup: Download Example Files

Download the example Serpent input and result files from the GitHub repository.

In [ ]:
import os
import urllib.request

# Create directory for examples
os.makedirs("Serpent", exist_ok=True)

# Base URL for raw GitHub content
base_url = "https://raw.githubusercontent.com/Funz/fz-Serpent/main/examples"

# Download example files
files_to_download = [
    ("Serpent/input.inp", f"{base_url}/Serpent/input.inp"),
    ("Serpent/input_res.m", f"{base_url}/Serpent/input_res.m"),
]

print("Downloading example files...")
for local_path, url in files_to_download:
    try:
        urllib.request.urlretrieve(url, local_path)
        print(f"  ✓ Downloaded: {local_path}")
    except Exception as e:
        print(f"  ✗ Failed to download {local_path}: {e}")

print("\n✓ Example files ready!")

## Import Required Libraries

In [ ]:
import fz
import serpentTools
import os
import tempfile
import json

print(f"fz version: {fz.__version__ if hasattr(fz, '__version__') else 'unknown'}")
print(f"serpentTools version: {serpentTools.__version__}")

## Example 1: Parse Input File for Variables

Use `fz.fzi()` to identify parameterized variables in a Serpent input file.

The Serpent input uses `${variable}` syntax for parameters like:
- `${enrichment}` - U-235 enrichment
- `${water_density}` - Coolant density
- `${fuel_radius}` - Fuel pellet radius

In [ ]:
# Parse the input file to find variables
variables = fz.fzi(
    "Serpent/input.inp",
    "Serpent"
)

print(f"Variables found in input.inp:")
for var in sorted(variables.keys()):
    print(f"  - {var}")

## Example 2: Compile Input Files

Use `fz.fzc()` to compile input files with specific parameter values. This substitutes variables with actual values.

In [ ]:
# Create temporary directory for compiled files
with tempfile.TemporaryDirectory() as tmpdir:
    # Compile input with specific values
    params = {
        "enrichment": 0.04,
        "u238_fraction": 0.8415,
        "water_density": 0.72,
        "water_temp": 600,
        "fuel_radius": 0.41,
        "clad_inner_radius": 0.42,
        "clad_outer_radius": 0.475,
        "pitch_half": 0.63,
        "neutrons_per_cycle": 10000,
        "active_cycles": 100,
        "inactive_cycles": 20,
        "seed": 12345
    }
    
    fz.fzc(
        "Serpent/input.inp",
        params,
        "Serpent",
        output_dir=tmpdir
    )
    
    # Find and show compiled file
    for root, dirs, files in os.walk(tmpdir):
        for f in files:
            if f.endswith(".inp"):
                compiled_file = os.path.join(root, f)
                print(f"Compiled file preview:")
                print("=" * 70)
                with open(compiled_file, 'r') as file:
                    lines = file.readlines()
                    # Show first 30 lines
                    for line in lines[:30]:
                        print(line, end='')
                print("\n... (truncated)")
                break

## Example 3: Parse Serpent Results with serpentTools

Demonstrate how serpentTools parses Serpent result files (`*_res.m`).

The fz-Serpent plugin extracts these outputs as JSON arrays:
- `absKeff`, `anaKeff`, `colKeff`, `impKeff` - k-effective estimates
- `burnup`, `burnDays` - Burnup data (for depletion calculations)

In [ ]:
# Parse the example result file
res = serpentTools.read("Serpent/input_res.m")

print("Serpent Results Summary")
print("=" * 70)

# Extract k-effective values
keff_data = res.resdata['absKeff']
print(f"\nAbsorption-based k-eff:")
print(f"  Value: {keff_data[0]:.5f}")
print(f"  Relative error: {keff_data[1]:.5f}")

# Show as JSON (how fz-Serpent exports it)
print(f"\nJSON output format:")
print(f"  absKeff: {json.dumps(keff_data.tolist())}")

# Check for burnup data
if 'burnup' in res.resdata:
    print(f"  burnup: {json.dumps(res.resdata['burnup'].tolist())}")
else:
    print(f"  burnup: [] (non-depletion case)")

if 'burnDays' in res.resdata:
    print(f"  burnDays: {json.dumps(res.resdata['burnDays'].tolist())}")
else:
    print(f"  burnDays: [] (non-depletion case)")

## Example 4: Running Parametric Study (Requires Serpent)

**Note:** This example requires a working Serpent installation.

The code below shows how to run a parametric enrichment study.

In [ ]:
# Define parameter values for parametric study
input_variables = {
    "enrichment": [0.03, 0.04, 0.05],  # U-235 enrichment
    "u238_fraction": [0.8515, 0.8415, 0.8315],  # Corresponding U-238
    "water_density": [0.72],
    "water_temp": [600],
    "fuel_radius": [0.41],
    "clad_inner_radius": [0.42],
    "clad_outer_radius": [0.475],
    "pitch_half": [0.63],
    "neutrons_per_cycle": [10000],
    "active_cycles": [100],
    "inactive_cycles": [20],
    "seed": [12345]
}

print(f"Parametric study configuration:")
print(f"  Enrichment values: {input_variables['enrichment']}")
print(f"\nNote: Running actual calculations requires Serpent installation.")

# Uncomment below to run actual calculations:
# results = fz.fzr(
#     input_path="Serpent/input.inp",
#     input_variables=input_variables,
#     model="Serpent",
#     calculators="localhost_Serpent",
#     results_dir="results/enrichment_study"
# )
# print(results[['enrichment', 'absKeff']])

## Example 5: Visualization

Example of how to visualize parametric study results.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example visualization with mock data
# (Replace with actual results from fz.fzr when running real calculations)

# Mock data: k-eff vs enrichment
enrichment = [0.02, 0.03, 0.04, 0.05, 0.06]
keff = [0.95, 1.05, 1.15, 1.25, 1.33]  # Typical trend
keff_err = [0.001, 0.001, 0.001, 0.001, 0.001]

plt.figure(figsize=(10, 6))
plt.errorbar(enrichment, keff, yerr=keff_err, fmt='o-', linewidth=2, 
             markersize=8, capsize=5, label='k-effective')
plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.7, label='Critical (k=1)')
plt.xlabel('U-235 Enrichment (wt%)', fontsize=12)
plt.ylabel('k-effective', fontsize=12)
plt.title('UOX Fuel Pin Cell: k-eff vs Enrichment', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Note: This plot uses mock data for demonstration.")
print("Run actual Serpent calculations to get real results.")

## Summary

This notebook demonstrated:

1. **Variable parsing** - Using `fz.fzi()` to identify parameters in Serpent input files
2. **Input compilation** - Using `fz.fzc()` to substitute parameter values
3. **Result parsing** - Using serpentTools to extract k-effective and burnup data
4. **Parametric studies** - Configuration for running enrichment sensitivity studies

For actual calculations, ensure you have:
- Serpent 2 installed and available in PATH
- Nuclear data libraries configured
- Sufficient computational resources